# Create inifiles Sophie

In [1]:
import glob, os
import pandas as pd

from configparser import ConfigParser
from pathlib import Path

## Set Paths

In [11]:
MODELDIR = Path('/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/')
FLUXDIR = Path('/lustre1/0/wtrcycle/users/jaerts/camels_wflow/sophie_BEP/')

flux_selection_file = 'fluxnet_toren_selectie_klein_Sophie_Jaren.csv'
config_file = "/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/global_05min/iniFileExample/setup_05min_non-natural.ini"

## Load FLUXNET selection DataFrame

In [12]:
df = pd.read_csv(f'{FLUXDIR}/{flux_selection_file}', sep=';')
df = df.set_index('SITE_ID')

In [13]:
df

,SITE_NAME,FLUXNET2015,FLUXNET-CH4,LOCATION_LAT,LOCATION_LONG,LOCATION_ELEV,IGBP,MAT,MAP,Aantal jaar,Klimaat,Vanaf,Tot,totaal
SITE_ID,,,,,,,,,,,,,,
AU-Cpr,Calperum,CC-BY-4.0,NaN,-34.00210,140.58910,NaN,SAV,NaN,NaN,5,BWk,2011-01,2013-12,3 jaar
BR-Npw,Northern Pantanal Wetland,NaN,CC-BY-4.0,-16.49800,-56.41200,120.0,WSA,24.90,1486.0,4,Aw,2015-07,2017-12,"1,5 jaar"
CN-Ha2,Haibei Shrubland,CC-BY-4.0,NaN,37.60860,101.32690,NaN,WET,NaN,NaN,3,Dwc,2003-01,2005-12,3 jaar
ES-LgS,Laguna Seca,CC-BY-4.0,NaN,37.09794,-2.96583,2267.0,OSH,NaN,NaN,3,Csb,2007-01,2008-12,2 jaar
FR-Pue,Puechabon,CC-BY-4.0,NaN,43.74130,3.59570,270.0,EBF,13.50,883.0,15,Csa,2008-01,2009-12,3 jaar
GF-Guy,Guyaflux (French Guiana),CC-BY-4.0,NaN,5.27877,-52.92486,48.0,EBF,25.70,3041.0,11,Am,2008-01,2010-12,3 jaar
MY-PSO,Pasoh Forest Reserve (PSO),CC-BY-4.0,NaN,2.97300,102.30620,NaN,EBF,NaN,NaN,7,Af,2007-01,2009-12,3 jaar
NL-Loo,Loobos,CC-BY-4.0,NaN,52.16658,5.74356,25.0,ENF,9.80,786.0,19,Cfb,2008-01,2010-12,3 jaar
SD-Dem,Demokeya,CC-BY-4.0,NaN,13.28290,30.47830,500.0,SAV,26.00,320.0,5,BWh,2008-01,2009-12,2 jaar


## Generate configuration files for each station

In [14]:
for station_id, row in df.iterrows():
    
    start_year = row.Vanaf[:4]
    end_year = row.Tot[:4]
    
    config = ConfigParser()
    config.optionxform = str 
    config.read(config_file)

    config['globalOptions']['outputDir'] = '/data/output'
    config['globalOptions']['inputDir'] = '/data/input'

    config['globalOptions']['cloneMap'] = f'global_05min/cloneMaps/SophieBEP/{station_id.lower()}_05min.map'

    config['globalOptions']['startTime'] = f'{start_year}-01-01'
    config['globalOptions']['endTime']   = f'{end_year}-12-31'

    config['meteoOptions']['referenceETPotMethod'] = 'Hamon'

    config['meteoOptions']['precipitationNC'] = f'global_05min/meteo/SophieBEP/pcrglobwb_OBS6_ERA5_reanaly_1_day_pr_{str((int(start_year) - 1))}-{end_year}_{station_id.lower()}.nc'
    config['meteoOptions']['temperatureNC'] = f'global_05min/meteo/SophieBEP/pcrglobwb_OBS6_ERA5_reanaly_1_day_tas_{str((int(start_year) - 1))}-{end_year}_{station_id.lower()}.nc'
    config['meteoOptions']['refETPotFileNC'] = f'global_05min/meteo/SophieBEP/pcrglobwb_OBS6_ERA5_reanaly_1_Eday_evspsblpot_{str((int(start_year) - 1))}-{end_year}_{station_id.lower()}.nc'
    
    config['meteoOptions']['precipitationVariableName'] = 'pr'
    config['meteoOptions']['temperatureVariableName'] = 'tas'
    config['meteoOptions']['referenceEPotVariableName'] = 'evspsblpot'
    
    config['meteoOptions']['precipitationConstant'] = '0.0'
    config['meteoOptions']['precipitationFactor'] = '1.0'
    config['meteoOptions']['temperatureConstant'] = '-273.15'
    config['meteoOptions']['temperatureFactor'] = '1.0'
    config['meteoOptions']['referenceEPotConstant'] = '0.0'
    config['meteoOptions']['referenceEPotFactor'] = '1.0'
    
    config['meteoDownscalingOptions']['downscalePrecipitation'] = 'True'
    config['meteoDownscalingOptions']['downscaleTemperature'] = 'True'
    config['meteoDownscalingOptions']['downscaleReferenceETPot'] = 'True'

    outfile = f'{MODELDIR}/{station_id.lower()}_05min_Hamon.ini'
    
    with open(outfile, 'w') as configfile:    # save
        config.write(configfile)
        print(configfile)

<_io.TextIOWrapper name='/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/au-cpr_05min_Hamon.ini' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/br-npw_05min_Hamon.ini' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/cn-ha2_05min_Hamon.ini' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/es-lgs_05min_Hamon.ini' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/fr-pue_05min_Hamon.ini' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/gf-guy_05min_Hamon.ini' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/my-pso_05min_Hamon.ini' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/nl-loo_05mi

In [15]:
for station_id, row in df.iterrows():
    
    start_year = row.Vanaf[:4]
    end_year = row.Tot[:4]
    
    config = ConfigParser()
    config.optionxform = str 
    config.read(config_file)

    config['globalOptions']['outputDir'] = '/data/output'
    config['globalOptions']['inputDir'] = '/data/input'

    config['globalOptions']['cloneMap'] = f'global_05min/cloneMaps/SophieBEP/{station_id.lower()}_05min.map'

    config['globalOptions']['startTime'] = f'{start_year}-01-01'
    config['globalOptions']['endTime']   = f'{end_year}-12-31'

    config['meteoOptions']['referenceETPotMethod'] = 'Input'

    config['meteoOptions']['precipitationNC'] = f'global_05min/meteo/SophieBEP/pcrglobwb_OBS6_ERA5_reanaly_1_day_pr_{str((int(start_year) - 1))}-{end_year}_{station_id.lower()}.nc'
    config['meteoOptions']['temperatureNC'] = f'global_05min/meteo/SophieBEP/pcrglobwb_OBS6_ERA5_reanaly_1_day_tas_{str((int(start_year) - 1))}-{end_year}_{station_id.lower()}.nc'
    config['meteoOptions']['refETPotFileNC'] = f'global_05min/meteo/SophieBEP/pcrglobwb_OBS6_ERA5_reanaly_1_Eday_evspsblpot_{str((int(start_year) - 1))}-{end_year}_{station_id.lower()}.nc'
    
    config['meteoOptions']['precipitationVariableName'] = 'pr'
    config['meteoOptions']['temperatureVariableName'] = 'tas'
    config['meteoOptions']['referenceEPotVariableName'] = 'evspsblpot'
    
    config['meteoOptions']['precipitationConstant'] = '0.0'
    config['meteoOptions']['precipitationFactor'] = '1.0'
    config['meteoOptions']['temperatureConstant'] = '-273.15'
    config['meteoOptions']['temperatureFactor'] = '1.0'
    config['meteoOptions']['referenceEPotConstant'] = '0.0'
    config['meteoOptions']['referenceEPotFactor'] = '1.0'
    
    config['meteoDownscalingOptions']['downscalePrecipitation'] = 'True'
    config['meteoDownscalingOptions']['downscaleTemperature'] = 'True'
    config['meteoDownscalingOptions']['downscaleReferenceETPot'] = 'True'

    outfile = f'{MODELDIR}/{station_id.lower()}_05min_Input.ini'
    
    with open(outfile, 'w') as configfile:    # save
        config.write(configfile)
        print(configfile)

<_io.TextIOWrapper name='/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/au-cpr_05min_Input.ini' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/br-npw_05min_Input.ini' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/cn-ha2_05min_Input.ini' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/es-lgs_05min_Input.ini' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/fr-pue_05min_Input.ini' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/gf-guy_05min_Input.ini' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/my-pso_05min_Input.ini' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/nl-loo_05mi